In [6]:
# install.packages("RMariaDB")

In [5]:
require('RMariaDB', quietly = T)

In [8]:
# Read the meta data
sanctumdbcon <- RMariaDB::dbConnect(
    drv = RMariaDB::MariaDB(),
    dbname = "Devdb",
    user = "admin",
    password = "JohnLennon1980",
    host = "sanctum-instance-1.cnklj0yzvfnv.us-east-1.rds.amazonaws.com",
    port = 3306
  )
clin_data <- dbReadTable(sanctumdbcon, name = clin_table)
dbDisconnect(sanctumdbcon)

ERROR: Error in dbReadTable(sanctumdbcon, name = clin_table): object 'clin_table' not found


In [ ]:
clin_data <- dcast(subset(clin_data, !is.na(key)), 
                 formula = model_id ~ key,
                 fun.aggregate = max) %>%
subset(., Model %in% colnames(counts)) # keep metadata for relavant samples

colnames(clin_data) <- gsub('..via.TruSight.clinical.', '', colnames(clin_data))
clin_data[] <- lapply(clin_data, toupper)
clin_data[clin_data=="PENDING" | clin_data=="NOT AVAILABLE" | clin_data=="OTHER" ] <- NA
clin_data[] <- lapply(clin_data, function(x) gsub("DIFFERENTIATED", "DIFF.", x))
clin_data[] <- lapply(clin_data, function(x) gsub("SQUAMOUS CELL", "SC", x))
# remove (largely) empty columns 
clin_data <- clin_data[apply(clin_data, 2, function(x) sum(is.na(x)) < nrow(clin_data)/2)]
# identify numeric columns
clin_data <- retype(clin_data)